In [ ]:
using DataFrames, MLJ

In [ ]:
X, y = @load_boston;
X = DataFrame(X)
y = Vector(y);

In [ ]:
the_model = @load RidgeRegressor pkg=MLJLinearModels add=true
the_pipe = @pipeline Standardizer the_model
the_machine = machine(the_pipe, X, y) |> fit!

In [ ]:
fitted_params(rrm).ridge_regressor

In [ ]:
report(rrm)